# RAG with Gemini and Qdrant: End-to-End Demo

This notebook demonstrates a Retrieval-Augmented Generation (RAG) workflow using Gemini (LLM) and Qdrant (vector database). It covers document ingestion, chunking, embedding, vector storage, retrieval, and answer generation.

## 1. Import Required Libraries
We import all necessary libraries for the RAG workflow, including Qdrant client, dotenv, and our utility modules.

In [ ]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from utils.chunk import chunk_text
from utils.embedding import embed_texts
from utils.gemini import ask_gemini

load_dotenv()

QDRANT_HOST = os.getenv("QDRANT_HOST", "localhost")
QDRANT_PORT = int(os.getenv("QDRANT_PORT", 6333))
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "rag_docs")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

## 2. Ingest a Text File
Read a sample text file from the `data/` folder, chunk it, embed the chunks, and store them in Qdrant.

In [ ]:
# Make sure you have a file at data/example.txt
file_path = "data/example.txt"

with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

chunks = chunk_text(text)
embeddings = embed_texts(chunks, api_key=GEMINI_API_KEY)
points = [PointStruct(id=i, vector=emb, payload={"text": chunk}) for i, (emb, chunk) in enumerate(zip(embeddings, chunks))]
client.upsert(collection_name=QDRANT_COLLECTION, points=points)
print(f"Ingested {len(points)} chunks into Qdrant.")

## 3. Query the RAG System
Ask a question, retrieve relevant chunks from Qdrant, and generate an answer using Gemini.

In [ ]:
question = "What is this document about?"  # Change as needed
top_k = 5
q_emb = embed_texts([question])[0]
hits = client.search(collection_name=QDRANT_COLLECTION, query_vector=q_emb, limit=top_k)
context = "\n".join([hit.payload["text"] for hit in hits])
answer = ask_gemini(question, context)
print("Context:\n", context)
print("\nAnswer:\n", answer)

## 4. (Optional) Inspect Chunks and Embeddings
You can inspect the generated chunks and their embeddings for debugging or exploration.

In [ ]:
# Show first 2 chunks and their embedding shape
for i, chunk in enumerate(chunks[:2]):
    print(f"Chunk {i}: {chunk[:100]}...")
    print(f"Embedding shape: {len(embeddings[i])}")